In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import time

from multiprocessing import Process, Queue, Lock, Manager

from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.metrics import dtw_subsequence_path

from signal import signal, SIGPIPE, SIG_DFL
signal(SIGPIPE, SIG_DFL) # ignore sigpipe

<Handlers.SIG_IGN: 1>

In [2]:
dataset_folder = "../all"
metadata_filename = "training_set_metadata.csv"
data_filename = "training_set.csv"
!ls {dataset_folder}

data_note.pdf	       test_set_metadata.csv  training_set_metadata.csv
sample_submission.csv  test_set_sample.csv
test_set.csv	       training_set.csv


In [3]:
metadata = pd.read_csv("{}/{}".format(dataset_folder, metadata_filename))
metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [4]:
object_ids = metadata.object_id.values
print("{} objects to predict".format(object_ids.size))

7848 objects to predict


In [5]:
def read_pickle(name):
    with open(name, 'rb') as f:
        data = pickle.load(f)
    return data

In [6]:
xg_model = read_pickle("xg_model")
shapelets = read_pickle("shapelets")
print(shapelets.shape)

(36,)


/home/frankl1/.env/plasticc/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
classes = np.array([ 6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, 99])
print("classes: ", classes)

classes:  [ 6 15 16 42 52 53 62 64 65 67 88 90 92 95 99]


In [8]:
def normalize(ts, ts_err):
    ts /= (ts_err + 1) # +1 to avoid zero division
    ts = np.nan_to_num(ts)
    ts = TimeSeriesScalerMinMax().fit_transform(ts)
    return ts

In [9]:
def distanceToShapelet(ts, shapelet):
    path, dist = dtw_subsequence_path(shapelet, ts)
    return dist

In [10]:
def distanceToShapelets(ts, shapelets=shapelets):
    return [distanceToShapelet(ts, shapelet) for shapelet in shapelets]

In [11]:
step = 1
nb_of_passband = 6
max_mjd = 1094
max_nb_of_processes = 3
print(nb_of_passband, max_mjd, sep="\n")

6
1094


In [12]:
def format_time_series(data):
    X = []
    X_err = []
    s0 = int(data.shape[0] / nb_of_passband)
    x, x_err = [], []
    for p in range(nb_of_passband):
        x = np.append(x, data[data.passband==p].flux.values)
        x_err = np.append(x_err, data[data.passband==p].flux_err.values)
    X.append(x.reshape(s0, nb_of_passband))
    X_err.append(x.reshape(s0, nb_of_passband))
    return np.array(X), np.array(X_err)

In [13]:
def mjdToPredict(mjdToExclude, minMjd, maxMjd, step):
    return np.array(list(set(range(minMjd, maxMjd+1, step)) - set(mjdToExclude)))

In [14]:
def fill_missing_flux(objectDf, maxMjd, step):
    objectDf.mjd = objectDf[["mjd"]].astype(np.int64)
    objectDf.mjd = objectDf.mjd - objectDf.mjd.min()
    instance = objectDf.groupby(["passband", "mjd"]).mean().reset_index()
    meanByPassband = instance.groupby("passband").mean()[["flux", "flux_err"]]
    frames = [instance]
    for p in range(6):
        toExclude = instance[instance.passband==p].mjd
        toPredict = mjdToPredict(toExclude, 0, maxMjd, step)
        nb = len(toPredict)
        frames.append(pd.DataFrame({
            'mjd': toPredict,
            'flux': [meanByPassband.loc[p].flux]*nb,
            'flux_err': [meanByPassband.loc[p].flux_err]*nb,
            'detected': [-1] * nb,
            'passband': [p]*nb,
            'object_id': [objectDf.object_id.values[0]]*nb
        }))
    return pd.concat(frames, sort=False)

In [15]:
def predict_object(test_df):
    instance = fill_missing_flux(test_df, max_mjd, step)
    ts, ts_err = format_time_series(instance)
    ts_norm = normalize(ts, ts_err)
    dist_vec = distanceToShapelets(ts_norm[0], shapelets)
    preds = xg_model.predict_proba([dist_vec])[0]
    preds = np.concatenate(([test_df.object_id.values[0]], preds, [0]))
    return preds

In [16]:
def producer_task(queue):
    data = pd.read_csv("{}/{}".format(dataset_folder, data_filename))
    for oid in object_ids[:20]:
#         print("Object {} produced".format(oid))
        object_df = data[data.object_id == oid]
        queue.put(object_df)

In [17]:
def consumer_task(queue, lock, submission): 
    # Run indefinitely
    while True:
        # If the queue is empty, queue.get() will block until the queue has data
        object_df = queue.get()
        preds = predict_object(object_df)
#         print("Object {} consumed".format(preds[0]))
        # Synchronize access to the console
        with lock:
            submission.append(preds)

In [18]:
def predict_test_set(predictions, nb_lines, max_nb_of_processes = 2):
    object_queue = Queue()

    lock = Lock()

    consumers = []

    producer = Process(target=producer_task, args=(object_queue,))

    for i in range(max_nb_of_processes):
        p = Process(target=consumer_task, args=(object_queue, lock, predictions))

        # This is critical! The consumer function has an infinite loop
        # Which means it will never exit unless we set daemon to true
        p.daemon = True
        consumers.append(p)

    producer.start()

    for c in consumers:
        c.start()

    len_preds = len(predictions)
    while len_preds < nb_lines:
        len_preds = len(predictions)
        print("{} / {} -> {} %".format(len_preds, nb_lines, np.round(len_preds * 100 / nb_lines if len_preds > 0 else 0)))
        time.sleep(2)
        
    print("{} %".format(np.round(len_preds * 100 / nb_lines if len_preds > 0 else 0)))
    print("Producer finished")

In [19]:
%%time
manager = Manager()

predictions = manager.list()

predict_test_set(predictions, 20, max_nb_of_processes=max_nb_of_processes)

0 / 20 -> 0 %
0 / 20 -> 0 %
9 / 20 -> 45.0 %
19 / 20 -> 95.0 %
20 / 20 -> 100.0 %
100.0 %
Producer finished
CPU times: user 46.2 ms, sys: 77.8 ms, total: 124 ms
Wall time: 10.2 s


In [20]:
columns = ["object_id"]
columns.extend([f"class_{c}" for c in classes])
submission = pd.DataFrame(columns = columns, data=list(predictions))
submission.object_id = submission.object_id.astype(np.int32)
print(submission.shape)

(20, 16)


In [21]:
submission.to_csv("submission.csv", index=False)